# Kurzschlussberechnung nach IEC 60909

In diesem Tutorial möchten wir Ihnen vorstellen, wie Sie in pandapower eine Kurzschlussberechnung umsetzen können. Hierbei wird gemäß IEC 60909 eine Kurzschlussberechnung derart umgesetzt, dass an der Fehlerstelle mit einer Ersatzspannungsquelle gerechnet wird. Umgesetzt ist dies für folgende Komponenten:
- sgen (as motor or as full converter generator)
- gen (as synchronous generator)
- ext_grid
- line
- trafo
- trafo3w
- impedance

mit den jeweiligen Korrekturfaktoren, die in IEC 60909 definiert sind. Lasten und Shunts werden, entsprechend der Norm, hierbei vernachlässigt. Das Schaltermodell ist komplett in die Berechnung übertragbar.

Folgende Kurzschlusströme werden berechnet:
- ikss (Anfangs-Kurzschlusswechselstrom)
- ip (Stoß-Kurzschlussstrom)
- ith (Thermisch gleichwertiger Kurzschlussstrom)

entweder als:
- symmetrisch dreiphasiger oder
- asymmetrisch zweiphasiger

Kurzschlussstrom. 

Berechnungen sind möglich für vermaschte und radiale Netze. ip und ith wurden nur implementiert für Kurzschlussströme weit von Synchrongeneratoren entfernt.

Alle Ergebnisse wurden gegenüber gängiger Softwaretools am Markt getestet.

## Beispielnetz

Um die Kurzschlussberechnung zu zeigen, definieren wir uns zuerste ein einfaches, offen-ringbetriebes Netz mit 4 Bussen, welche untereinander über zwei Leitungen und einem Transformator verbunden sind. Ein offner Schalter gewährleistet, dass das Netz unvermascht, offen betrieben wird.

<img src="shortcircuit/example_network_sc.png">

In [ ]:
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)
    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)
    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

## Symmetrische Kurzschlussberechnung

### Maximalkurzschlussströme

Als erstes berechnen wir die maximalen Kurzschlussströme:

In [ ]:
net = ring_network()
sc.calc_sc(net, case='max', ip=True, ith=True, branch_results=True)
net.res_bus_sc

Für Kanten werden die Ergebnisse als die maximal auftretenden Kurzschlussströme bestimmt, die auftreten, sobald ein Fehler an einem beliebigen Bus im Netz auftritt. Die Ergebnisse sind einerseits verfügbar für Leitungen ...:

In [ ]:
net.res_line_sc

... sowie für Transformatoren:

In [ ]:
net.res_trafo_sc

### Minimalkurzschlussströme
Ähnlich wie im Falle der Maximalkurzschlussströme können auch die Minimalkurzschlussströme berechnet werden. Allerdings müssen in diesem Fall die Endtemperaturen der Leitungen im Falle eines Fehlers gemäß der Norm angegeben werden.

In [ ]:
net = ring_network()
net.line["endtemp_degree"] = 80
sc.calc_sc(net, case="min", ith=True, ip=True, branch_results=True)
net.res_bus_sc

Entsprechend werden dann auch die Minimalkurzschlusströme in den einzelnen Kantenelementen ausgegeben:

In [ ]:
net.res_line_sc

In [ ]:
net.res_trafo_sc

### Asynchronmaschinen


Asynchonmaschinen können dadurch erzeugt werden, dass man bei der Definierung eines statischen Generators einfach den Typ "Motor" angbit. Wichtig hierbei ist, dass man für die Kurzschluss-Impedanz einen R/X-Verhältnis, in pandapower "rx" bezeichnet, sowie das Verhältnis zwischen Nominalstrom und Kurzschlussstrom spezifiziert. Dieser Wert ist in pandapower als "k" definiert.

In [ ]:
net = ring_network()
pp.create_sgen(net, 2, p_mw=0, sn_mva=0.5, k=1.2, rx=7., type="motor")
net

Führen wir dieses Mal eine Kurzschlussberechnung durch, so können wir feststellen, dass die Ströme angestiegen sind. 

In [ ]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

### Synchronmaschinen

Synchronmaschinen sind in pandapower durch das "gen"-Element erfasst. Gemäß der Norm sind für die Berechnung des Kurzschlussstromes der Leistungsfaktor (cos$\varphi$) "cos_phi", die Nennspannung "vn_kv", die Nennscheinleistung "sn_mva" sowie der substransiente Widerstand "rdss" und die substransiente Reaktanz "xdss" von Nöten, um die Kurzschlussimpedanz zu berechnen: 

In [ ]:
net = ring_network()
pp.create_gen(net, 2, p_mw=0, vm_pu=1.0, cos_phi=0.8, vn_kv=22, sn_mva=5, xdss_pu=0.2, rdss_pu=0.005)
net

Führen wir erneut die Kurzschlussberechnung durch, so erhalten wir nun folgendes Ergebnis:

In [ ]:
sc.calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

Wie schon beim Asynchronmotor steigt der Kurzschlussstrom durch den Generator/Motor. Allerdings kann man an der Warnung erkennen, dass nur für weite Entfernungen des Fehlers die Ergebnisse korrekt sind.

### Vermaschte Netze

Die Korrekturfaktoren für aperiodische und thermische Ströme unterscheiden sich zwischen vermaschten und radialen Netzen.

In unserem Fall laden wir wieder das Radialnetz zu Beginn, schließen dieses Mal jedoch den Schalter am Ende, um so ein vermaschtes Netz zu erzeugen:

In [ ]:
net = ring_network()
net.switch.closed = True
sc.calc_sc(net, ip=True, ith=True)
net.res_bus_sc

Zur Validierung vergleichen wir an dieser Stelle nochmals die Ergebnisse mit den Ergebnisse des Radialnetzes. Hierzu öffnen wir den Schalter wieder:

In [ ]:
net.switch.closed = False
sc.calc_sc(net, ip=True, ith=True)
net.res_bus_sc

Des Weiteren werfen wir einen Blick auf die Ströme innerhalb der Kanten unter der Annahme, dass das Netz vermascht betrieben wird:

In [ ]:
net.switch.closed = True
sc.calc_sc(net, topology="auto", ip=True, ith=True, branch_results=True)
net.res_line_sc

Man kann erkennen, dass die Ströme um einiges kleiner sind als an den Busen. Dies liegt daran, dass sich die Ströme gleichmäßig auf die beiden Halbringe verteilen können.

### Fault Impedance

Des Weiteren ist es möglich eine Fehlerimpedanz zu definieren ...:

In [ ]:
net = ring_network()
sc.calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.)

... was die Kurzschlussströme weiter verringert:

In [ ]:
net.res_bus_sc

## Asymmetrische zweiphasige Kurzschlussberechnungen

Alle Berechnung von oben können auch als zweiphasige Kurzschlussberechnung durchgeführt werden, indem man einfach als "fault"-Wert "2ph" angibt:

In [ ]:
net = ring_network()
sc.calc_sc(net, fault="2ph", ip=True, ith=True)
net.res_bus_sc

Häufig wird diese Annahme für Minimalkurzschlussstromberechnungen verwendet:

In [ ]:
net = ring_network()
net.line["endtemp_degree"] = 150
sc.calc_sc(net, fault="2ph", case="min", ip=True, ith=True)
net.res_bus_sc

## Lessons learned

Am Ende dieses Tutorials sind Sie in der Lage:
- eine dreiphasige Kurzschlussberechnung sowie 
- eine zweiphasige Kurzschlussberechnung 

mit pandapower durchzuführen.

Sie haben zudem verstanden, welche Auswirkungen:
- Maschen
- Synchronmaschinen
- Asynchronmaschinen

auf die Berechnungen haben.